In [1]:
from time import sleep
import google.generativeai as genai
import configparser
import json
import re
import pandas as pd

/home/gcasado0/proyectos/chatbot/chatbot/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [28]:
# leer archivo config.ini
config = configparser.ConfigParser()
config.read('config.ini')
GOOGLE_API_KEY = config['GOOGLE']['GOOGLE_API_KEY']
genai.configure(api_key=GOOGLE_API_KEY)
# MODELO= 'gemini-1.5-flash'
MODELO= 'gemini-2.0-flash'
model = genai.GenerativeModel(MODELO)

In [29]:
def analizar_consulta(consulta: str, url_result: str, temperatura=0.2):
  prompt = f"""
  Recibirás consultas enviadas por vecinos al sitio web de la municipalidad.

  Tu tarea es analizar el contenido del mensaje y una lista de palabras de la URL de origen (proporcionada entre triple comillas) y devolver la siguiente información en formato JSON:

  1. **Sentimiento general** del texto (por ejemplo: positivo, negativo, neutral).
  2. **¿El remitente expresa enojo?** (Sí o No).
  3. **Categoría principal** de la consulta, elegida de la siguiente lista:
    - Reclamos
    - Multas
    - Licencia de Conducir
    - Impuestos
    - Perfil Digital
    - Salud
    - Indefinido

  4. **Categoría secundaria**, según la categoría principal elegida:

  - **Reclamos**:
    - Alumbrado
    - Arbolado
    - Higiene urbana
    - Indefinido

  - **Multas**:
    - Pagar
    - Turnos
    - Titularidad
    - Descargos
    - Indefinido

  - **Licencia de Conducir**:
    - Turnos
    - Consultas
    - Cenat y/o Sellados
    - Indefinido

  - **Impuestos**:
    - TGI
    - Patente
    - Indefinido

  - **Salud**:
    - Vacunas
    - Sin turnos
    - Error en el sistema
    - Reclamo
    - Indefinido

  - **Perfil Digital**:
    - Actualización de datos
    - Error en el sistema
    - Indefinido

  Consulta del vecino:
  '''{consulta}'''

  Lista de palabras de la URL de origen:
  '''{url_result}'''

  Devuelve exclusivamente la siguiente salida en formato JSON:
    "Sentimiento": "<positivo | negativo | neutral>",
    "Enojo": "<Sí | No>",
    "Categoría Principal": "<una de las categorías principales>",
    "Categoría Secundaria": "<una de las categorías secundarias posibles según la principal>"
  """

  response = model.generate_content(
          prompt,
          generation_config = genai.GenerationConfig(
          temperature=0.2,
          )
      )

  return response.text

In [32]:
url_result ="inicio/usuariomr"
consulta = """
"Buenas tardes, soy Extranjero Venezolano y antes no tenía DNI y me otorgaron el CUIL/CUIT anses que tienen actualmente, pero, ya me llego el DNI y actualice mi CUIL/CUIT
Datos actuales:
  DNI: 96372499
  CUIL/CUIT: 27963724999

COMO PODRÁ DARSE CUENTA NECESITO QUE MI CUENTA SE ACTUALICE CON MIS DOCUMENTOS ACTUALIZADOS 
"""

url_result = "obtener carnet manipulacion alimentos"
consulta ="No me deja sacar turno."

print(analizar_consulta(consulta, url_result))

```json
{
  "Sentimiento": "negativo",
  "Enojo": "No",
  "Categoría Principal": "Salud",
  "Categoría Secundaria": "Sin turnos"
}
```


In [45]:
data = pd.read_excel('datos/encontraste_lo_que_buscabas.xlsx')
data.head(5)

,Serial number,Submission ID,Creado,Página actual,Remote IP address,Notes,Email,Mensaje,URL,respuesta
0,166305,2109849,2024-03-26 00:01:08,NaN,186.13.96.200,NaN,ivanagutierrez92@gmail.com,NaN,https://www.rosario.gob.ar/inicio/gestionesmr?...,si
1,166306,2109857,2024-03-26 00:03:42,NaN,186.152.252.13,NaN,NaN,NaN,https://www.rosario.gob.ar/inicio/obtener-el-c...,si
2,166307,2109858,2024-03-26 00:05:14,NaN,181.117.29.235,NaN,NaN,NaN,https://www.rosario.gob.ar/inicio/agenda/busca...,si
3,166308,2109866,2024-03-26 00:09:54,NaN,181.169.48.96,NaN,monicazrzn@hotmail.com,Ya respondí que ese auto no figura a mi nombre...,https://www.rosario.gob.ar/inicio/infraccion/0...,comentario
4,166309,2109882,2024-03-26 00:21:33,NaN,186.13.96.162,NaN,julianjoseabrt@gmail.com,NaN,https://www.rosario.gob.ar/inicio/usuariomr?co...,si


In [46]:
# eliminar del dataset los registros con la columna respuesta = "si"
data2 = data[data['respuesta'] != "si"].reset_index(drop=True)

In [47]:
data2

,Serial number,Submission ID,Creado,Página actual,Remote IP address,Notes,Email,Mensaje,URL,respuesta
0,166308,2109866,2024-03-26 00:09:54,NaN,181.169.48.96,NaN,monicazrzn@hotmail.com,Ya respondí que ese auto no figura a mi nombre...,https://www.rosario.gob.ar/inicio/infraccion/0...,comentario
1,166314,2109918,2024-03-26 00:48:01,NaN,186.111.148.79,NaN,emiricci1984@gmail.com,No pude hacer el descargo,https://www.rosario.gob.ar/inicio/avisosmrdeta...,comentario
2,166315,2109920,2024-03-26 00:48:19,NaN,186.13.96.95,NaN,juansuclla@gmail.com,"Hola, no puedo sacar un turno para la renovaci...",https://www.rosario.gob.ar/inicio/turnosmr,no
3,166318,2109952,2024-03-26 01:42:35,NaN,179.60.219.84,NaN,martinezfito31@gmail.com,Renovación del medio boleto estudiantil,https://www.rosario.gob.ar/inicio/renovar-medi...,no
4,166319,2109959,2024-03-26 01:51:34,NaN,179.60.219.84,NaN,martinezfito31@gmail.com,Renovación de medio boleto estudiantil de Delf...,https://www.rosario.gob.ar/inicio/renovar-medi...,comentario
...,...,...,...,...,...,...,...,...,...,...
23307,226542,3068431,2024-12-20 07:50:28,NaN,186.64.82.134,NaN,marvale17@gmail.com,No está el hospital centenario,https://www.rosario.gob.ar/inicio/centros-de-s...,no
23308,226543,3068434,2024-12-20 07:58:43,NaN,181.199.145.46,NaN,sonmividaaa@gmail.com,Turno para sacar TGI,https://www.rosario.gob.ar/inicio/buscar%3Fpal...,no
23309,226544,3068453,2024-12-20 08:25:09,NaN,181.92.101.124,NaN,robertoapesato2@gmail.com,¿Donde esta la solapa GESTIONES para revisar e...,https://www.rosario.gob.ar/inicio/,no
23310,226548,3068477,2024-12-20 08:45:55,NaN,186158144146,NaN,edu_gorba@hotmail.com,Necesito hacer el seguimiento sobre el trámite...,https://www.rosario.gob.ar/inicio/openid-conne...,no


In [48]:
import re
from urllib.parse import urlparse

# Lista básica de palabras vacías en español
STOPWORDS_ES = {
    "el", "la", "los", "las", "de", "del", "al", "y", "o", "en", "un", "una", "por", "para", "a"
}

def extraer_palabras_clave(url):
    # Extraer path de la URL
    parsed_url = urlparse(url)
    path = parsed_url.path

    # Tomar el último segmento del path
    ultimo_segmento = path.strip("/").split("/")[-1]

    # Reemplazar guiones por espacios y eliminar caracteres no alfabéticos
    limpio = re.sub(r'[^a-zA-ZáéíóúñüÁÉÍÓÚÑÜ\- ]', '', ultimo_segmento.replace("-", " ")).lower()

    # Separar en palabras y quitar stopwords
    palabras = [p for p in limpio.split() if p and p not in STOPWORDS_ES]

    # Devolver como string
    return " ".join(palabras)



In [ ]:
# recorrer todo el dataframe data y extraer el resumen, temas, sentimientos, enojo, complejidad y no sabe actividad

for i, row in data2.iterrows():

    consulta = data2['Mensaje'][i]
    url_origen = data2['URL'][i]
    # si la consulta es nula o el largo de la consulta es menor a 10 caracteres no la proceso
    if pd.isnull(consulta) or len(consulta) < 10:
        continue 

    url_result = extraer_palabras_clave(url_origen)

    print(i, url_result)
    print(consulta)
    
    respuesta=analizar_consulta(consulta, url_result)
    # Imprimir la respuesta              
    print(respuesta)
    # Extraer el contenido JSON del Markdown
    markdown_text = respuesta

    # Usar una expresión regular para extraer el contenido JSON
    json_match = re.search(r'```json\n(.*?)\n```', markdown_text, re.DOTALL)
    if json_match:
        json_string = json_match.group(1)
        try:
            # Convertir la cadena de texto a un objeto JSON
            analisis = json.loads(json_string)
            # Agregar respuesta al dataframe data por cada campo de response.text
            data2.loc[i, 'Sentimiento'] = analisis.get('Sentimiento', 'None')
            data2.loc[i, 'Enojo'] = analisis.get('Enojo', 'None')
            data2.loc[i, 'Principal'] = analisis.get('Categoría Principal', 'None')
            data2.loc[i, 'Secundaria'] = analisis.get('Categoría Secundaria', 'None')            

        except json.JSONDecodeError as e:
            print(f"Error al decodificar JSON: {e}")
    else:
        print("No se encontró contenido JSON en la respuesta.")
        
    sleep(5) # para no exceder la cuota de Google de 15 requests por minuto
            
    if i>10:
        break

# guardar el dataframe data con las respuestas en un archivo Excel
fecha_hora= pd.Timestamp.now().strftime('%Y%m%d_%H%M%S')
# eliminar del dataset los registros con la columna Sentimiento = None
data2 = data2[data2['Sentimiento'].notna()]
data2.to_excel(f"datos/encontraste_lo_que_buscabas-{MODELO}-{fecha_hora}.xlsx", index=False)

3 bea fc f bed dcfbf
Ya respondí que ese auto no figura a mi nombre,fue vendido hace más de 12 años y figura en Rosario yo vendí a Roberto Martin
```json
{
  "Sentimiento": "neutral",
  "Enojo": "No",
  "Categoría Principal": "Impuestos",
  "Categoría Secundaria": "Patente"
}
```
9 
No pude hacer el descargo
```json
{
  "Sentimiento": "negativo",
  "Enojo": "No",
  "Categoría Principal": "Multas",
  "Categoría Secundaria": "Descargos"
}
```
10 turnosmr
Hola, no puedo sacar un turno para la renovación  del carnet de conducir. Gracias
```json
{
  "Sentimiento": "neutral",
  "Enojo": "No",
  "Categoría Principal": "Licencia de Conducir",
  "Categoría Secundaria": "Turnos"
}
```
13 renovar medio boleto estudiantil
Renovación del medio boleto estudiantil
```json
{
  "Sentimiento": "neutral",
  "Enojo": "No",
  "Categoría Principal": "Indefinido",
  "Categoría Secundaria": "Indefinido"
}
```
14 renovar medio boleto estudiantil
Renovación de medio boleto estudiantil de Delfina Martínez, D.N.I